In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.wait import WebDriverWait
from pathlib import Path


from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException 
from selenium.webdriver.support import expected_conditions as EC
import os
import requests
import time
import shutil
import glob
import configparser



In [2]:
url = "https://bball-index.shinyapps.io/Lebron/__sockjs__/n=1X3EzJCDf7I2hrdArE/t=aafccad80f857164d4ddfc525f26a6b7/w=d2402a9b/s=0/396/gvz1w1g2/eventsource"
headers = {
      
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",

        "Connection": "keep-alive",
    }
page = requests.get(url,headers = headers)
soup = BeautifulSoup(page.content, "html.parser")
soup.find('table')

In [ ]:
soup.text

In [ ]:
def scrape_LEBRON():
    headers = {
      
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",

        "Connection": "keep-alive",
    }
    url = 'https://www.bball-index.com/lebron-database/'
    page = requests.get(url,headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    table = soup.find(id = 'table_1')
    df = pd.read_html(str(table),displayed_only=False)[0]
    df = df.dropna(subset = 'Player')
    year = df['Season'].str[0:4].astype(int) +1
    df['year'] = year
    df['Player'] = df['Player'].str.lower()
    df = df.rename(columns = {'Team':'team','LEBRON Contract Value':'Value Added'})
    return df
df = scrape_LEBRON()

df.to_csv('lebron.csv',index = False)
df = df[df['Season'] =='2023-24']
df.to_csv('lebron/lebron.csv',index = False)
#df.to_csv('2023/lebron/lebron.csv',index = False)

In [88]:
old = pd.read_csv('old_lebron.csv')
new = pd.read_csv('lebron.csv')
new

,bref_id,Player,Season,team,Offensive Archetype,Minutes,Games,Pos,Age,LEBRON,O-LEBRON,D-LEBRON,WAR,Value Added,NBA ID,boxLEBRON,boxOLEBRON,boxDLEBRON,year,Defensive Role
0,hillgr01,grant hill,2009-10,PHO,Athletic Finisher,2430,81.0,SF,37,-0.46,-0.82,0.36,3.0,4443027.0,255.0,0.66,0.52,0.14,2010,NaN
1,hunteli01,lindsey hunter,2009-10,CHI,Secondary Ball Handler,122,13.0,PG,39,-1.00,-0.24,-0.76,0.1,156535.0,283.0,-0.69,-0.35,-0.34,2010,NaN
2,onealsh01,shaquille o'neal,2009-10,CLE,Post Scorer,1240,53.0,C,37,0.29,-1.37,1.65,2.2,3209036.0,406.0,0.97,-0.28,1.25,2010,NaN
3,howarju01,juwan howard,2009-10,POR,Post Scorer,1632,73.0,C,36,-3.49,-2.19,-1.29,-1.3,-1943330.0,436.0,-2.43,-2.01,-0.42,2010,NaN
4,kiddja01,jason kidd,2009-10,DAL,Stationary Shooter,2881,80.0,PG,36,2.87,1.24,1.63,10.0,14937060.0,467.0,3.27,1.81,1.46,2010,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7570,colliza01,zach collins,2023-24,SAS,Versatile Big,812,31.0,C,26,-1.61,-1.34,-0.27,0.4,1480736.0,1628380.0,-2.06,-1.39,-0.68,2024,NaN
7571,lavinza01,zach lavine,2023-24,CHI,Shot Creator,635,18.0,SG,29,0.23,0.67,-0.44,1.1,3703583.0,203897.0,0.35,0.74,-0.40,2024,NaN
7572,nnajize01,zeke nnaji,2023-24,DEN,Roll + Cut Big,301,29.0,PF,23,-2.01,-2.02,0.00,0.1,281553.0,1630192.0,-1.60,-1.89,0.28,2024,NaN
7573,willizi02,ziaire williams,2023-24,MEM,Movement Shooter,569,31.0,SF,22,-2.69,-2.31,-0.39,-0.1,-297104.0,1630533.0,-2.33,-2.24,-0.09,2024,NaN


In [89]:
old = old[old.year==2024]
old = old[['Player','Defensive Role']]
new = new[new.year==2024]
new['Player'] = new['Player'].str.replace('.','', regex=True)

name_dict = dict(zip(new['Player'],new['NBA ID']))
old['NBA ID'] = old['Player'].map(name_dict)

new.drop(columns='Defensive Role',inplace = True)
master = new.merge(old,how='left')
master

C:\Users\gaber\AppData\Local\Temp\ipykernel_14648\2623897461.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['Player'] = new['Player'].str.replace('.','', regex=True)
C:\Users\gaber\AppData\Local\Temp\ipykernel_14648\2623897461.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.drop(columns='Defensive Role',inplace = True)


,bref_id,Player,Season,team,Offensive Archetype,Minutes,Games,Pos,Age,LEBRON,O-LEBRON,D-LEBRON,WAR,Value Added,NBA ID,boxLEBRON,boxOLEBRON,boxDLEBRON,year,Defensive Role
0,lawsoaj01,aj lawson,2023-24,DAL,Stationary Shooter,127,13.0,SG,23,-0.44,-0.31,-0.13,0.2,552934.0,1630639.0,-0.88,-0.56,-0.32,2024,Wing Stopper
1,greenaj01,aj green,2023-24,MIL,Stationary Shooter,151,21.0,SG,24,-1.60,-0.51,-1.10,0.1,276210.0,1631260.0,-1.03,-0.02,-1.02,2024,Chaser
2,griffaj01,aj griffin,2023-24,ATL,Movement Shooter,110,12.0,SF,20,-1.18,-1.03,-0.15,0.1,301676.0,1631100.0,-1.24,-0.95,-0.30,2024,Helper
3,gordoaa01,aaron gordon,2023-24,DEN,Shot Creator,920,29.0,PF,28,0.94,0.39,0.55,2.0,6807928.0,203932.0,0.47,0.15,0.32,2024,Wing Stopper
4,holidaa01,aaron holiday,2023-24,HOU,Secondary Ball Handler,525,29.0,PG,27,-2.21,-1.57,-0.64,0.1,271494.0,1628988.0,-2.19,-1.54,-0.65,2024,Chaser
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,colliza01,zach collins,2023-24,SAS,Versatile Big,812,31.0,C,26,-1.61,-1.34,-0.27,0.4,1480736.0,1628380.0,-2.06,-1.39,-0.68,2024,Anchor Big
515,lavinza01,zach lavine,2023-24,CHI,Shot Creator,635,18.0,SG,29,0.23,0.67,-0.44,1.1,3703583.0,203897.0,0.35,0.74,-0.40,2024,Helper
516,nnajize01,zeke nnaji,2023-24,DEN,Roll + Cut Big,301,29.0,PF,23,-2.01,-2.02,0.00,0.1,281553.0,1630192.0,-1.60,-1.89,0.28,2024,Mobile Big
517,willizi02,ziaire williams,2023-24,MEM,Movement Shooter,569,31.0,SF,22,-2.69,-2.31,-0.39,-0.1,-297104.0,1630533.0,-2.33,-2.24,-0.09,2024,Point of Attack


In [90]:
old

,Player,Defensive Role,NBA ID
7553,nikola jokic,Anchor Big,203999.0
7554,shai gilgeous-alexander,Helper,1628983.0
7555,luka doncic,Helper,1629029.0
7556,giannis antetokounmpo,Wing Stopper,203507.0
7557,joel embiid,Mobile Big,203954.0
...,...,...,...
8059,bilal coulibaly,Wing Stopper,1641731.0
8060,brandon miller,Helper,1641706.0
8061,corey kispert,Helper,1630557.0
8062,malaki branham,Chaser,1631103.0


In [92]:
master[master['Defensive Role'].isnull()]

,bref_id,Player,Season,team,Offensive Archetype,Minutes,Games,Pos,Age,LEBRON,O-LEBRON,D-LEBRON,WAR,Value Added,NBA ID,boxLEBRON,boxOLEBRON,boxDLEBRON,year,Defensive Role
7,sanogad01,adama sanogo,2023-24,CHI,Low Minute,9,2.0,PF,22,-0.14,-0.26,0.12,0.0,47623.0,1641766.0,-0.28,-0.20,-0.08,2024,NaN
50,willibr03,brandon williams,2023-24,DAL,Low Minute,7,1.0,PG,24,-0.53,-0.35,-0.18,0.0,27994.0,1630314.0,-0.39,-0.29,-0.10,2024,NaN
185,badjiib01,ibou badji,2023-24,POR,Roll + Cut Big,50,4.0,C,21,-0.57,-0.84,0.28,0.1,203796.0,1630641.0,-0.44,-0.65,0.20,2024,NaN
302,loftoke01,kenneth lofton jr,2023-24,MEM/PHI,NaN,102,0.0,PF,0,0.07,-0.36,0.44,0.2,562848.0,1631254.0,0.04,-0.26,0.30,2024,NaN
306,johnske07,keon johnson,2023-24,BRK,Low Minute,21,1.0,SG,22,-0.49,-0.41,-0.08,0.0,88913.0,1630553.0,-0.58,-0.60,0.02,2024,NaN
435,councri01,ricky council iv,2023-24,PHI,Low Minute,6,1.0,SG,22,-1.82,-0.71,-1.11,0.0,8147.0,1641741.0,-1.68,-0.57,-1.11,2024,NaN
479,maledth01,theo maledon,2023-24,CHO,NaN,200,0.0,PG,0,-1.00,-0.72,-0.28,0.2,628846.0,1630177.0,-0.95,-0.84,-0.11,2024,NaN
491,queentr01,trevelin queen,2023-24,ORL,Low Minute,1,1.0,SG,27,-0.50,-0.17,-0.34,0.0,5292.0,1630243.0,-0.54,-0.16,-0.38,2024,NaN


In [93]:
master['Defensive Role'] = master['Defensive Role'].fillna('Low Minute')

In [94]:
master

,bref_id,Player,Season,team,Offensive Archetype,Minutes,Games,Pos,Age,LEBRON,O-LEBRON,D-LEBRON,WAR,Value Added,NBA ID,boxLEBRON,boxOLEBRON,boxDLEBRON,year,Defensive Role
0,lawsoaj01,aj lawson,2023-24,DAL,Stationary Shooter,127,13.0,SG,23,-0.44,-0.31,-0.13,0.2,552934.0,1630639.0,-0.88,-0.56,-0.32,2024,Wing Stopper
1,greenaj01,aj green,2023-24,MIL,Stationary Shooter,151,21.0,SG,24,-1.60,-0.51,-1.10,0.1,276210.0,1631260.0,-1.03,-0.02,-1.02,2024,Chaser
2,griffaj01,aj griffin,2023-24,ATL,Movement Shooter,110,12.0,SF,20,-1.18,-1.03,-0.15,0.1,301676.0,1631100.0,-1.24,-0.95,-0.30,2024,Helper
3,gordoaa01,aaron gordon,2023-24,DEN,Shot Creator,920,29.0,PF,28,0.94,0.39,0.55,2.0,6807928.0,203932.0,0.47,0.15,0.32,2024,Wing Stopper
4,holidaa01,aaron holiday,2023-24,HOU,Secondary Ball Handler,525,29.0,PG,27,-2.21,-1.57,-0.64,0.1,271494.0,1628988.0,-2.19,-1.54,-0.65,2024,Chaser
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,colliza01,zach collins,2023-24,SAS,Versatile Big,812,31.0,C,26,-1.61,-1.34,-0.27,0.4,1480736.0,1628380.0,-2.06,-1.39,-0.68,2024,Anchor Big
515,lavinza01,zach lavine,2023-24,CHI,Shot Creator,635,18.0,SG,29,0.23,0.67,-0.44,1.1,3703583.0,203897.0,0.35,0.74,-0.40,2024,Helper
516,nnajize01,zeke nnaji,2023-24,DEN,Roll + Cut Big,301,29.0,PF,23,-2.01,-2.02,0.00,0.1,281553.0,1630192.0,-1.60,-1.89,0.28,2024,Mobile Big
517,willizi02,ziaire williams,2023-24,MEM,Movement Shooter,569,31.0,SF,22,-2.69,-2.31,-0.39,-0.1,-297104.0,1630533.0,-2.33,-2.24,-0.09,2024,Point of Attack


In [95]:
orig =pd.read_csv('lebron.csv')
orig = orig[orig.year!=2024]
orig

,bref_id,Player,Season,team,Offensive Archetype,Minutes,Games,Pos,Age,LEBRON,O-LEBRON,D-LEBRON,WAR,Value Added,NBA ID,boxLEBRON,boxOLEBRON,boxDLEBRON,year,Defensive Role
0,hillgr01,grant hill,2009-10,PHO,Athletic Finisher,2430,81.0,SF,37,-0.46,-0.82,0.36,3.0,4443027.0,255.0,0.66,0.52,0.14,2010,NaN
1,hunteli01,lindsey hunter,2009-10,CHI,Secondary Ball Handler,122,13.0,PG,39,-1.00,-0.24,-0.76,0.1,156535.0,283.0,-0.69,-0.35,-0.34,2010,NaN
2,onealsh01,shaquille o'neal,2009-10,CLE,Post Scorer,1240,53.0,C,37,0.29,-1.37,1.65,2.2,3209036.0,406.0,0.97,-0.28,1.25,2010,NaN
3,howarju01,juwan howard,2009-10,POR,Post Scorer,1632,73.0,C,36,-3.49,-2.19,-1.29,-1.3,-1943330.0,436.0,-2.43,-2.01,-0.42,2010,NaN
4,kiddja01,jason kidd,2009-10,DAL,Stationary Shooter,2881,80.0,PG,36,2.87,1.24,1.63,10.0,14937060.0,467.0,3.27,1.81,1.46,2010,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7051,iveyja01,jaden ivey,2022-23,DET,Primary Ball Handler,2304,74.0,SG,21,-3.72,-1.69,-2.03,-1.8,-5823127.0,1631093.0,-2.46,-0.79,-1.67,2023,Helper
7052,thorjt01,jt thor,2022-23,CHO,Athletic Finisher,969,69.0,PF,20,-3.72,-3.40,-0.32,-0.8,-2452117.0,1630550.0,-3.21,-3.08,-0.12,2023,Helper
7053,holidju01,justin holiday,2022-23,ATL/DAL,Movement Shooter,706,46.0,SG,34,-3.72,-2.65,-1.07,-0.6,-1787899.0,203200.0,-2.02,-1.64,-0.38,2023,Chaser
7054,lopezro01,robin lopez,2022-23,CLE,Post Scorer,299,37.0,C,35,-4.17,-3.14,-1.03,-0.3,-1013626.0,201577.0,-3.10,-2.37,-0.73,2023,Anchor Big


In [96]:
to_save = pd.concat([orig,master])
to_save

,bref_id,Player,Season,team,Offensive Archetype,Minutes,Games,Pos,Age,LEBRON,O-LEBRON,D-LEBRON,WAR,Value Added,NBA ID,boxLEBRON,boxOLEBRON,boxDLEBRON,year,Defensive Role
0,hillgr01,grant hill,2009-10,PHO,Athletic Finisher,2430,81.0,SF,37,-0.46,-0.82,0.36,3.0,4443027.0,255.0,0.66,0.52,0.14,2010,NaN
1,hunteli01,lindsey hunter,2009-10,CHI,Secondary Ball Handler,122,13.0,PG,39,-1.00,-0.24,-0.76,0.1,156535.0,283.0,-0.69,-0.35,-0.34,2010,NaN
2,onealsh01,shaquille o'neal,2009-10,CLE,Post Scorer,1240,53.0,C,37,0.29,-1.37,1.65,2.2,3209036.0,406.0,0.97,-0.28,1.25,2010,NaN
3,howarju01,juwan howard,2009-10,POR,Post Scorer,1632,73.0,C,36,-3.49,-2.19,-1.29,-1.3,-1943330.0,436.0,-2.43,-2.01,-0.42,2010,NaN
4,kiddja01,jason kidd,2009-10,DAL,Stationary Shooter,2881,80.0,PG,36,2.87,1.24,1.63,10.0,14937060.0,467.0,3.27,1.81,1.46,2010,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,colliza01,zach collins,2023-24,SAS,Versatile Big,812,31.0,C,26,-1.61,-1.34,-0.27,0.4,1480736.0,1628380.0,-2.06,-1.39,-0.68,2024,Anchor Big
515,lavinza01,zach lavine,2023-24,CHI,Shot Creator,635,18.0,SG,29,0.23,0.67,-0.44,1.1,3703583.0,203897.0,0.35,0.74,-0.40,2024,Helper
516,nnajize01,zeke nnaji,2023-24,DEN,Roll + Cut Big,301,29.0,PF,23,-2.01,-2.02,0.00,0.1,281553.0,1630192.0,-1.60,-1.89,0.28,2024,Mobile Big
517,willizi02,ziaire williams,2023-24,MEM,Movement Shooter,569,31.0,SF,22,-2.69,-2.31,-0.39,-0.1,-297104.0,1630533.0,-2.33,-2.24,-0.09,2024,Point of Attack


In [97]:

to_save.to_csv('lebron.csv',index = False)

In [ ]:

old['year_id'] = old['Player'].astype(str)+old['year'].astype(str)
print(old['year_id'])
droles = dict(zip(old['year_id'],old['Defensive Role']))
droles

In [ ]:
files = glob.glob('epm/*')
try:
    for f in files:
        os.remove(f)
    os.remove(r'C:\Users\gaber\Downloads\EPM data.csv')
except Exception as e:
    print('No file located, proceeding') 
    
config = configparser.RawConfigParser()
config.read('conf.cfg')
    
details_dict = dict(config.items('LOGIN'))


username = details_dict['username']
password = details_dict['password']


def load_epm(username,password):
    options=webdriver.ChromeOptions()
    # Step 1: Create a session and load the page
    epm = 'https://dunksandthrees.com/epm'
    directory =  str(os.getcwd()) +'\epm'
    pref={"download.default_directory":directory}
    #example: prefs = {"download.default_directory" : "C:\Tutorial\down"};
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)
    driver.get(epm)
    try:
        # find username/email field and send the username itself to the input field
        driver.find_element("name", "email").click()
        driver.find_element("name", "email").send_keys(username)
        # find password input field and insert password as well
        driver.find_element("name", "password").click()
        driver.find_element("name", "password").send_keys(password)
        # click login button
        xpath ='/html/body/div/div[2]/div/div/div[2]/form/div[2]/div[2]/button'
        wait = WebDriverWait(driver, 10)
        pause = wait.until(EC.presence_of_element_located((By.XPATH,xpath)))

        driver.find_element(By.XPATH, xpath).click()
        
        path2 ='/html/body/div/main/div/div[2]/div[1]/div[6]/span/button'
        #wait = WebDriverWait(driver, 10)
        #pause = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'outlined svelte-aai53c'))) 
        time.sleep(5)
        driver.find_element(By.XPATH, path2).click()
        print('located')

        
        #url_list = [url1,url2,url3,url4,url5]

        #os.remove('epm/epm.csv')
        time.sleep(10)
        #os.rename('epm/EPM data.csv','epm/epm.csv')
        driver.close()
        
    except Exception as e:
        print(e)

load_epm(username,password)

In [ ]:
download_path = r'C:\Users\gaber\Downloads\EPM data.csv'
#replace download path with whatever the file path for your download folder is
df = pd.read_csv(download_path)
df.to_csv('epm/epm.csv',index = False)

In [ ]:
'''
for i in range(2013,2022):
        


 
    path = str(i+1)+'/lebron/'
    output_dir = Path(path)
    output_dir.mkdir(parents=True, exist_ok=True)
    frame = df[df['year'] ==i+1]
    frame.to_csv(str(i+1)+'/lebron/lebron.csv',index = False)'''

In [ ]:
df